source: https://www.kaggle.com/szmnkrisz97/point-to-uncertainty-different-ranges-per-level

In [1]:
import sys
import os
import pathlib
import gc
import pandas as pd
pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_rows', 500)
import numpy as np
import math
import random
import pickle
import time
import psutil
import seaborn as sns
import matplotlib.pyplot as plt

# custom import
import scipy.stats  as stats

# constant variables for helper functions

In [2]:
N_CORES = psutil.cpu_count()     # Available CPU cores
print(f"N_CORES: {N_CORES}")

N_CORES: 12


# function nicely diplaying a head of Pandas DataFrame

In [3]:
import IPython

def display(*dfs, head=True):
    for df in dfs:
        IPython.display.display(df.head() if head else df)

# function fixing random seeds

In [4]:
def seed_everything(seed=0):
    """Sets seed to make all processes deterministic     # type: int
    
    """
    random.seed(seed)
    np.random.seed(seed)

SEED = 42
seed_everything(SEED)    

# function processing df in multiprocess

In [5]:
def run_df_in_multiprocess(func, t_split):
    """Process ds in Multiprocess
    
    """
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

# other helper functions

In [6]:
def get_memory_usage():
    """メモリ使用量を確認するためのシンプルな「メモリプロファイラ」
    
    """
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)


def merge_by_concat(df1, df2, merge_on):
    """
    dtypesを失わないための連結による結合
    
    """
    
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1


#  constant variables for data import

In [7]:
# take the data from M5_accuracy
_DATA_DIR = os.path.sep.join(["data", "M5_Three_shades_of_Dark_Darker_magic"])
_OUTPUT_UNCERTAINTY_DIR = os.path.sep.join(["data", "Point_to_uncertainty_different_ranges_per_level"])
_OUTPUT_DIR = _OUTPUT_UNCERTAINTY_DIR

_CALENDAR_CSV_FILE = "calendar.csv"
_SAMPLE_SUBMISSION_CSV_FILE = "sample_submission.csv"
_SALES_TRAIN_VALIDATION_CSV_FILE = "sales_train_validation.csv"
_SELL_PRICES_CSV_FILE = "sell_prices.csv"

# _ACCURACY_DATA_DIR = os.path.sep.join(["data", "M5_Three_shades_of_Dark_Darker_magic", "bk", "0.47353"])
_ACCURACY_RESULT_FILE = "submission_v5_evaluation.csv"

# import data

In [8]:
def reduce_mem_usage(df, verbose=True):
    """
    reduce the memory usage of the given dataframe.
    https://qiita.com/hiroyuki_kageyama/items/02865616811022f79754
    
    Args:
        df: Dataframe
        verbose: 
        
    Returns:
        df, whose memory usage is reduced.

    Raises:
        None
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #columns毎に処理
        col_type = df[col].dtypes
        if col_type in numerics: #numericsのデータ型の範囲内のときに処理を実行. データの最大最小値を元にデータ型を効率的なものに変更
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def read_csv_data(directory, file_name):
    print('Reading files...')
    df = pd.read_csv(os.path.sep.join([str(directory), _DATA_DIR, file_name]))
    df = reduce_mem_usage(df)
    print('{} has {} rows and {} columns'.format(file_name, df.shape[0], df.shape[1]))
    
    return df

# read csv data

In [9]:
parent_dir = pathlib.Path(os.path.abspath(os.curdir)).parent
print(f"parent_dir: {parent_dir}")

df_sales_train_validation = read_csv_data(parent_dir, _SALES_TRAIN_VALIDATION_CSV_FILE)

parent_dir: /Users/kensaku-okada-mac/jdsc/Documents/study meeting/ds/okada_presentation_202008/aws_sagemaker_forecast/M5_Forecasting
Reading files...


FileNotFoundError: [Errno 2] File /Users/kensaku-okada-mac/jdsc/Documents/study meeting/ds/okada_presentation_202008/aws_sagemaker_forecast/M5_Forecasting/data/M5_Three_shades_of_Dark_Darker_magic/sales_train_validation.csv does not exist: '/Users/kensaku-okada-mac/jdsc/Documents/study meeting/ds/okada_presentation_202008/aws_sagemaker_forecast/M5_Forecasting/data/M5_Three_shades_of_Dark_Darker_magic/sales_train_validation.csv'

In [ ]:
# import accuracy result
df_accuracy_result = read_csv_data(parent_dir, _ACCURACY_RESULT_FILE)
df_accuracy_result = reduce_mem_usage(df_accuracy_result)



In [ ]:
# df_sample_submission = read_csv_data(parent_dir, _SAMPLE_SUBMISSION_CSV_FILE)

In [ ]:
# print(f"df_sales_train_validation: {df_sales_train_validation.head()}")
# print(f"df_sample_submission: {df_sample_submission.head()}")
# print(f"df_sales_train_validation: {df_sales_train_validation}")
# print(f"df_accuracy_result: {df_accuracy_result}")

# merge the dataframe

In [ ]:
# matches only "*_validation"
sub = df_accuracy_result.merge(df_sales_train_validation[["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]], on = "id")
sub["_all_"] = "Total"
print(f"sub.shape: {sub.shape}")
print(f"sub: {sub}")

# Different ratios for different aggregation levels
The higher the aggregation level, the more confident we are in the point prediction --> lower coef, relatively smaller range of quantiles

In [ ]:
qs = np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995])

def get_ratios(coef=0.15):
    qs2 = np.log(qs/(1-qs))*coef
#     print(f"qs2: {qs2}")
    
#     累積密度分布(cumulative distribution function)
    ratios = stats.norm.cdf(qs2)
#     print(f" ratios: { ratios}")
    
#     ratios[4] is 0.5
    ratios /= ratios[4]
#     print(f" ratios: { ratios}")
    
    ratios = pd.Series(ratios, index=qs)
#     return ratios.round(3)
    return ratios


In [ ]:
# coef between 0.05 and 0.24 is used, probably suboptimal values for now: <-グリッドサーチ手作りでベストなcoefの組み合わせ出せる？
level_coef_dict = {"id": get_ratios(coef=0.3), "item_id": get_ratios(coef=0.15),
                   "dept_id": get_ratios(coef=0.08), "cat_id": get_ratios(coef=0.07),
                   "store_id": get_ratios(coef=0.08), "state_id": get_ratios(coef=0.07), "_all_": get_ratios(coef=0.05),
                   ("state_id", "item_id"): get_ratios(coef=0.19),  ("state_id", "dept_id"): get_ratios(coef=0.1),
                    ("store_id","dept_id") : get_ratios(coef=0.11), ("state_id", "cat_id"): get_ratios(coef=0.08),
                    ("store_id","cat_id"): get_ratios(coef=0.1)
                  }

print(f"level_coef_dict: {level_coef_dict}")

For the the lowest level (i.e. "id") (30490 series), the smallest and biggest quantiles are 20% and 180% of the point prediction. For categories ("cat_id": 3 series), the model will be way more confident: the smallest quantile will be 71%, the biggest will be 129% of the point prediction.
    
    

In [ ]:
def quantile_coefs(quantiles, level):
#     特定レベルのquantile ratioを取得
    ratios = level_coef_dict[level]

#     各probablity intervalの閾値(quantiles)に対する倍率を取得
    quantile_values = ratios.loc[quantiles].values
#     print(f"quantile_values: {quantile_values}")
#     print(f"quantile_values[:, None]: {quantile_values[:, None]}")
    
    return quantile_values

def get_group_preds(pred, level, cols):
#     levelごとにcols(各Fの値)を合計
    df = pred.groupby(level)[cols].sum()
    print(f"df.shape: {df.shape}")
        
    q = np.repeat(qs, len(df))    
    print(f"q.shape: {q.shape}")
    print(f"q at get_group_preds: {q}")
    
#     quantileの数は9
    df = pd.concat([df]*9, axis=0, sort=False)
    df.reset_index(inplace = True)
    print(f"amplified df: {df}")

#     accuracyにおける予測値の予測区間を、倍率を掛けることにより計算。[:, None]で行持ちを列持ちに変換
    df[cols] *= quantile_coefs(q, level)[:, None]

    if level != "id":
#         uncertainty 用の提出ファイルに合わせるためのlabelの変更
        df["id"] = [f"{lev}_X_{q:.3f}_validation" for lev, q in zip(df[level].values, q)]
    else:
        df["id"] = [f"{lev.replace('_validation', '')}_{q:.3f}_validation" for lev, q in zip(df[level].values, q)]
    
    
    df = df[["id"]+list(cols)]
    return df

def get_couple_group_preds(pred, level1, level2):
    df = pred.groupby([level1, level2])[cols].sum()
    print(f"df.shape: {df.shape}")

    q = np.repeat(qs, len(df))
    df = pd.concat([df]*9, axis=0, sort=False)
    df.reset_index(inplace = True)
        
    df[cols] *= quantile_coefs(q, (level1, level2))[:, None]
    df["id"] = [f"{lev1}_{lev2}_{q:.3f}_validation" for lev1,lev2, q in 
                zip(df[level1].values,df[level2].values, q)]
    df = df[["id"]+list(cols)]
    return df

In [ ]:
levels = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "_all_"]
couples = [("state_id", "item_id"),  ("state_id", "dept_id"),("store_id","dept_id"), ("state_id", "cat_id"),("store_id","cat_id")]
cols = [f"F{i}" for i in range(1, 29)]

df = []
for level in levels :
    df.append(get_group_preds(sub, level, cols))
    
for level1,level2 in couples:
    df.append(get_couple_group_preds(sub, level1, level2))

print(f"appended df: {df}")

# 縦方向に連結
df = pd.concat(df, axis=0, sort=False)

# inplace=Trueで元のオブジェクトを直接変更
df.reset_index(drop=True, inplace=True)

# validationとevaluation行　それぞれにコピーする(同じ値が格納される)
df_submission = pd.concat([df,df] , axis=0, sort=False)
df_submission.reset_index(drop=True, inplace=True)
# 後半部分のid名を"_evaluation"に変更
df_submission.loc[df_submission.index >= len(df_submission.index)//2, "id"] = df_submission.loc[df_submission.index >= len(df_submission.index)//2, "id"].str.replace(
                                    "_validation$", "_evaluation")

print(f"df_submission: {df_submission}")

# substitute the calculation result to the submission format

In [ ]:
parent_dir = pathlib.Path(os.path.abspath(os.curdir)).parent

# Reading competition sample submission and merging our predictions
submission_df = pd.read_csv(os.path.sep.join([str(parent_dir), _OUTPUT_UNCERTAINTY_DIR, _SAMPLE_SUBMISSION_CSV_FILE]))
submission_df = reduce_mem_usage(submission_df)

submission_ids_df = submission_df[["id"]]

# submission_df = pd.read_csv(ORIGINAL+_SAMPLE_SUBMISSION_CSV_FILE)[['id']]
my_submission_df = submission_ids_df.merge(df, on=['id'], how='left').fillna(0)
print(f"my_submission_df:{my_submission_df}")

# export train/test result as csv

In [ ]:
parent_dir = pathlib.Path(os.path.abspath(os.curdir)).parent

VER = 5

_EXPORT_FILE_NAME = 'submission_v'+str(VER)+'_validation.csv'
print("csv data export start")
my_submission_df.to_csv(os.path.sep.join([str(parent_dir), _OUTPUT_DIR, _EXPORT_FILE_NAME]), index=False)
print('csv data export finished. Size:', my_submission_df.shape)